In [ ]:
# RECURRUENT INPUT값을 다음의 INPUT값으로 활용하는것(SEQUENTIAL 한 데이터 이용할때)
# A에는 총 3개의 W가 있따. INPUT때 곱하는 W(xh), H구할때 곱하는 W(hh), 자신한테 RECURSIVE 되는 W(hy).
# 바닐라, 가장 기본이되는것(one to one: 하나를 입력해서 하나르 뱉는), one to many(sequence), many to one..

#colah'blog 가 잘설명해놓음 rnnm, lstm http://colah.github.io/posts/2015-08-Understanding-LSTMs/

#dnn 은 입력값하나당 하나의 아웃풋만 있기 떄문에 'hello'의 예측값을 맞출때 학습하지 못함
# rnn 은 hel, e, hell 뭘 입력해도 모두 잘 예측
# 구글 자동완성

#tesorflowKR에서 코드보면서 연습하는것도 개꿀임 TensorFlowKR/awesome_tensorflow_implementations

#waston tone analizer 를 통해 sentimental analysis 가능, 사진표정분석도 가능 

In [ ]:
# 보통 rnn을 그냥 쓰기보다는 LSTM과 GRU를 쓴다(게이트연산을 추가한다)
# 이전에 들어온값에 가중치를 줄여라, 지금 입력값에 가중치를 올려라(WEIGHT를 조정가능하기 때문에 더 선호)


In [1]:
import numpy as np
import tensorflow as tf

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# hi hello 
# X : hihell , Y : ihello
idx2char = ['h','i', 'e', 'l', 'o']
x_data = [[0,1,0,2,3,3] ]
x_one_hot = [[
                [1,0,0,0,0],
                [0,1,0,0,0],
                [1,0,0,0,0],
                [0,0,1,0,0],
                [0,0,0,1,0],
                [0,0,0,1,0]
]]
y_data = [[1,0,2,3,3,4]]

In [8]:
x_one_hot

[[[1, 0, 0, 0, 0],
  [0, 1, 0, 0, 0],
  [1, 0, 0, 0, 0],
  [0, 0, 1, 0, 0],
  [0, 0, 0, 1, 0],
  [0, 0, 0, 1, 0]]]

In [3]:
# shape(batch_size, sequence_length, input_dim)
num_classes = 5
input_dim = 5 #input으로 들어가는 값의 dimension
hidden_size = 5 #hidden node의 갯수
batch_size = 1 #한번에 들어가는 input 갯수
sequence_length = 6  #한번에 들어가는 sequence의 렝스 hihello(6개)
learning_rate = 0.1

In [4]:
X = tf.placeholder(tf.float32, shape = [None, sequence_length, input_dim])
Y = tf.placeholder(tf.int32, shape = [None, sequence_length])

# cell = tf.contrib.rnn.BasicRNNCell(num_units = hidden_size, state_is_tuple=True) #state 값을 튜플로 사용할거냐
cell = tf.contrib.rnn.BasicLSTMCell(num_units = hidden_size, state_is_tuple=True) 
#tuple을 사용하면 consatant 처럼 변화하지 않기 때문에 사용함(list, dictionary는 변함)
# cell = tf.contrib.rnn.BasicGRUCell(num_units = hidden_size, state_is_tuple=True) 

initial_state = cell.zero_state(batch_size, tf.float32) #다 0으로 채우는 함수, 어짜피 학습이되며 바뀜
outputs, _states = tf.nn.dynamic_rnn(cell, X, initial_state=initial_state, dtype=tf.float32)

X_for_fc = tf.reshape(outputs, [-1,hidden_size]) #flatten 시킴
outputs = tf.contrib.layers.fully_connected(inputs = X_for_fc, num_outputs = num_classes, 
                                            activation_fn = None) 
#sequence loss를 계산하기위해 reshape를 다시해야함, 예측 후 다시 변환

outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])
weights = tf.ones([batch_size, sequence_length])
sequence_loss = tf.contrib.seq2seq.sequence_loss(logits = outputs, targets= Y, weights = weights )# weight를 통해시퀀스의 가중치를 정해주기

loss = tf.reduce_mean(sequence_loss)
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)
prediction = tf.argmax(outputs,axis=2)

In [5]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
for step in range(4000):
    l,_ = sess.run([loss,train], feed_dict={X:x_one_hot, Y:y_data})
    result = sess.run(prediction, feed_dict={X:x_one_hot})
    if step % 100 == 0:
        print ("Step:",step,"Loss:",l,"Prediction:", result)
        result_str = [idx2char[c] for c in np.squeeze(result)]
        print ("Prediction:","".join(result_str))


Step: 0 Loss: 1.5841112 Prediction: [[3 3 3 3 3 3]]
Prediction: llllll
Step: 100 Loss: 0.0018113748 Prediction: [[1 0 2 3 3 4]]
Prediction: ihello
Step: 200 Loss: 0.00088647817 Prediction: [[1 0 2 3 3 4]]
Prediction: ihello
Step: 300 Loss: 0.0005425431 Prediction: [[1 0 2 3 3 4]]
Prediction: ihello
Step: 400 Loss: 0.00036939638 Prediction: [[1 0 2 3 3 4]]
Prediction: ihello
Step: 500 Loss: 0.00026893718 Prediction: [[1 0 2 3 3 4]]
Prediction: ihello
Step: 600 Loss: 0.0002050969 Prediction: [[1 0 2 3 3 4]]
Prediction: ihello
Step: 700 Loss: 0.0001617331 Prediction: [[1 0 2 3 3 4]]
Prediction: ihello
Step: 800 Loss: 0.00013082298 Prediction: [[1 0 2 3 3 4]]
Prediction: ihello
Step: 900 Loss: 0.000107977445 Prediction: [[1 0 2 3 3 4]]
Prediction: ihello
Step: 1000 Loss: 9.057475e-05 Prediction: [[1 0 2 3 3 4]]
Prediction: ihello
Step: 1100 Loss: 7.706559e-05 Prediction: [[1 0 2 3 3 4]]
Prediction: ihello
Step: 1200 Loss: 6.625811e-05 Prediction: [[1 0 2 3 3 4]]
Prediction: ihello
Step: 13